In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/machine learning/datasets/CO2 Emissions_Canada.csv')

In [ ]:
dataset = dataset.drop(['Model','Make'],axis = 1)

In [ ]:
dataset

In [ ]:
dataset.describe()

In [ ]:
dataset.isnull().sum()

In [ ]:
unique_items = dataset['Fuel Type'].unique()
print(unique_items)
print(len(unique_items))

In [ ]:
dataset.dtypes

In [ ]:
unique_items = dataset['Transmission'].unique()
print(unique_items)
print(len(unique_items))

In [ ]:
dataset.shape

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers = [('encoder',OneHotEncoder(sparse = False,drop = 'first'),['Fuel Type','Transmission','Vehicle Class'])],remainder = 'passthrough')
ct.fit_transform

In [ ]:
X = dataset.drop(['CO2 Emissions(g/km)'],axis = 1)
y = dataset.iloc[:,-1]

In [ ]:
X.shape

In [ ]:
X = ct.fit_transform(X)

In [ ]:
X.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X[:,45:51] = ss.fit_transform(X[:,45:51])

In [ ]:
Xy = np.concatenate((X, y.values.reshape(-1, 1)), axis=1)

In [ ]:
Xy.shape

In [ ]:
# prompt: calculate explain variance for Xy and plot a graph

from sklearn.decomposition import PCA
pca = PCA()
pca.fit(Xy)
plt.plot(np.cumsum(pca.explained_variance_ratio_), marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance')
plt.show()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 30)
X = pca.fit_transform(X)

In [ ]:
ds_1 = pd.DataFrame(X)
ds_1

In [ ]:
ds_1.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state = 4)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_train.shape)
print(y_test.shape)

In [ ]:
#random forest
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 15, random_state = 0)
regressor.fit(X_train, y_train)
y_predrd = regressor.predict(X_test)
np.set_printoptions(precision=2)
print(np.concatenate((y_predrd.reshape(len(y_predrd),1), y_test.values.reshape(len(y_test),1)),1))

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, y_predrd)

In [ ]:
y_train_pred = regressor.predict(X_train)

# Predictions on the testing set
y_test_pred = regressor.predict(X_test)

# Calculate R-squared for training and testing sets
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

# Calculate the overfitting score
overfitting_score = r2_train - r2_test

print(f"R-squared (Training): {r2_train}")
print(f"R-squared (Testing): {r2_test}")
print(f"Overfitting Score: {overfitting_score}")

In [ ]:
# prompt: find the r2_score after applying k cross validation

from sklearn.model_selection import cross_val_score

# Perform k-fold cross-validation
scores = cross_val_score(regressor, X, y, cv=5, scoring='r2')

# Calculate the average R-squared score
average_r2 = np.mean(scores)

# Print the average R-squared score
print(f"Average R-squared (k-fold CV): {average_r2}")


In [ ]:
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt

# Perform cross-validated predictions
cv_predictions = cross_val_predict(regressor, X, y, cv=5)

# Plot actual vs. predicted values
plt.scatter(y, cv_predictions)
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Actual vs. Predicted Values (Cross-Validation)")
plt.show()


In [ ]:
# Assuming your model is rf_model
feature_importances = regressor.feature_importances_
print("Feature Importances:", feature_importances)


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Assuming y_pred is your model predictions on the validation set
mae = mean_absolute_error(y_test, y_predrd)
mse = mean_squared_error(y_test, y_predrd)
rmse = np.sqrt(mse)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)


In [ ]:
residuals = y_test - y_predrd
plt.scatter(y_predrd, residuals)
plt.xlabel("Predicted Values")
plt.ylabel("Residuals")
plt.show()